# Data Split

In [ ]:
import os
import random
import numpy as np
from tqdm import tqdm

is_synthetic = True
pid_list = os.listdir("./Dataset/physionet.org/files/eicu-crd/2.0/preprocessed/sequences")
random.seed(1234)
random.shuffle(pid_list)

X, Y = [], []
for i, pid in enumerate(tqdm(pid_list)):
    if i<int(0.8*len(pid_list)) and is_synthetic==True:
        seq = np.load(f"./synthetic_dataset/sequences/{pid}")
        label = np.load(f"./synthetic_dataset/labels/{pid}")
    else:
        seq = np.load(f"./Dataset/physionet.org/files/eicu-crd/2.0/preprocessed/sequences/{pid}")
        label = np.load(f"./Dataset/physionet.org/files/eicu-crd/2.0/preprocessed/labels/{pid}")
    
    X.append(seq)
    Y.append(label)
    
X = np.stack(X, axis=0)
Y = np.stack(Y, axis=0)

X_train, Y_train = X[:int(0.8*len(X))], Y[:int(0.8*len(X))]
X_val, Y_val = X[int(0.8*len(X)):int(0.9*len(X))], Y[int(0.8*len(X)):int(0.9*len(X))]
X_test, Y_test = X[int(0.9*len(X)):], Y[int(0.9*len(X)):]

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

for i in range(5):
    print(i)
    clf = RandomForestClassifier(max_depth=3, random_state=0)
    clf.fit(X_train.reshape(-1, 48), Y_train[:, i])
    
    print(roc_auc_score(Y_train[:, i], clf.predict_proba(X_train.reshape(-1, 48))[:,1]))
    print(roc_auc_score(Y_val[:, i], clf.predict_proba(X_val.reshape(-1, 48))[:,1]))
    print(roc_auc_score(Y_test[:, i], clf.predict_proba(X_test.reshape(-1, 48))[:,1]))
    print()